In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql as pymysql
from urllib.parse import quote
import requests
import json
from urllib3.exceptions import InsecureRequestWarning

In [ ]:
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
ZABBIX_API_URL = "http://192.168.10.109/zabbix/api_jsonrpc.php"
UNAME = "Admin"
PWORD = "zabbix"

In [ ]:
readFile = pd.read_csv('./host_macro.csv',sep=";")

In [ ]:
def Auth():
    r = requests.post(ZABBIX_API_URL,
                  json={
                      "jsonrpc": "2.0",
                      "method": "user.login",
                      "params": {
                          "user": UNAME,
                          "password": PWORD},
                      "id": 1
                  }, verify=False)
    AUTHTOKEN = r.json()["result"]
    return AUTHTOKEN


In [ ]:
def host_get(host):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "host.get",
        "params": {
            "filter": {
                "host": [
                    str(host)
                ]
            }
        },
        "auth": Auth(),
        "id": 1
    }, verify=False)
    host_id = ''
    for value in r.json()['result']:
        host_id = value['hostid']
        
    return host_id

In [ ]:
def macro_get(host):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "host.get",
        "params": {
            "filter": {
                "host": [
                    str(host)
                ]
            }
        },
        "auth": Auth(),
        "id": 1
    }, verify=False)
    print(json.dumps(r.json(), indent=4, sort_keys=True))
    host_id = ''
    for value in r.json()['result']:
        host_id = value['macro']
        
    return host_id

In [ ]:
def macro_exist(host, macro):
    r = requests.post(ZABBIX_API_URL, json={
         "jsonrpc": "2.0",
        "method": "usermacro.get",
        "params": {
            "output": "extend",
            "hostids": str(host)
        },
        "auth": Auth(),
        "id": 1
    }, verify=False)
    for i in r.json()['result']:
        if i['hostid'] == host and i['macro'] == macro:
            r = requests.post(ZABBIX_API_URL, json={
                "jsonrpc": "2.0",
                "method": "usermacro.delete",
                "params": [
                    str(i['hostmacroid'])
                ],
                "auth": Auth(),
                "id": 1
            }, verify=False)

In [ ]:
def create_macro(host, macro, value):
    r = requests.post(ZABBIX_API_URL,json={
        "jsonrpc": "2.0",
        "method": "host.massadd",
        "params": {
            "hosts": [
                {
                    "hostid": str(host),
                }
            ],
            "macros": [
                {
                    "macro": str(macro),
                    "value": str(value)
                }
            ]
        },
        "auth": Auth(),
        "id": 1
    }, verify=False)
    print('Add Macro in HostID [', host,']','Macro/Value [',macro,' <-> ',value,'] Successfully')

In [ ]:
def mass_macro():
    group_list = pd.DataFrame()
    group_list['hostname'] = readFile['hostname']
    group_list['macro'] = readFile['macro'].str.split('|')
    group_list['value_macro'] = readFile['value_macro'].str.split('|')
    df_result = group_list.set_index(['hostname']).apply(pd.Series.explode).reset_index()
    for x,y in df_result.iterrows():
        host_id = host_get(y['hostname'])
        requests.post(ZABBIX_API_URL,json={
            "jsonrpc": "2.0",
            "method": "host.massadd",
            "params": {
                "hosts": [
                    {
                        "hostid": str(host_id),
                    }
                ],
                "macros": [
                    {
                        "macro": str(y['macro']),
                        "value": str(y['value_macro'])
                    }
                ]
            },
            "auth": Auth(),
            "id": 1
        }, verify=False)
        print('Add Mass Macro in Hostname [', y['hostname'],']','Macro/Value [',y['macro'],' <-> ',y['value_macro'],'] Successfully')

In [ ]:
def insert_macro():
    for index, row in readFile.iterrows():
        try:
            host_id = host_get(row['hostname'])
            macro = row['macro'].split('|')[0]
            value = row['value_macro'].split('|')[0]
            macro_exist(host_id, macro)
            create_macro(host_id,macro,value)
        except:
            continue

In [ ]:
if __name__ == '__main__':
    # insert_macro()
    mass_macro()